# Fretchet Inception Distance
**FID** measures the distance between the **Inception-v3 activation distributions** for **generated** and **real** samples.

But before looking into FID let's discuss a little about it's predecessor Inception Score.

### Inception Score
The inception score computes the KL divergence between the conditional class distribution and the marginal class distribution.It measures the quality of generated images and their diversity.

Let's first talk about entropy in the sense of a random variable or a probability distribution. Entropy can be 
viewed as randomness. If the value of a random variable x is highly predictable, it has low entropy. On the
contrary, if it is highly unpredictable, the entropy is high.Now let's look at the equation to compute IS.

1. Conditional Probability: 
   
   We want the conditional probability P(y|x) to be highly predictable (low entropy). i.e. given an image, we should know the object type easily. So we use an Inception network to classify the generated images and predict P(y|x) — where y is the label and x is the generated data. This reflects the quality of the images.
2. Marginal Probability: 
   
   The Marginal probability is computed as P(y)=If the generated images are diverse, the data distribution for y should be uniform (high entropy).
   
One shortcoming for IS is that it can misrepresent the performance if it only generates one image per class. p(y) will still be uniform even though the diversity is low.


**FID** is a more principled and comprehensive **metric** and has been shown to be more consistent with *human evaluation in assessing the realism and variation of the generated samples*.

The calculation can be divided into three parts:

- We **use the Inception network to extract 2048-dimensional activations from the pool3 layer for real and generated samples respectively**.
- Then we model the data distribution for these features using a multivariate Gaussian distribution with **mean µ** and **covariance Σ**. The `calculate_activation_statistics()` function does this.
- Finally **Wasserstein-2 distance is calculated for the mean and covariance of real images(x) and generated images(g)**.


`calculate_fretchet_distance()` function does this.

**Lower FID means better quality and diversity.**

| **Summary**

The **FID** metric calculates the distance between two distributions of images.
Typically, we have summary statistics (mean & covariance matrix) of one
of these distributions, while the 2nd distribution is given by a GAN.

In [21]:

from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import matplotlib.animation as animation
from IPython.display import HTML

from scipy import linalg
from torch.nn.functional import adaptive_avg_pool2d

from PIL import Image

import os
import pathlib

import numpy as np
import torch
from scipy import linalg
from torch.nn.functional import adaptive_avg_pool2d

from PIL import Image

try:
    from tqdm import tqdm
except ImportError:
    # If not tqdm is not available, provide a mock version of it
    def tqdm(x): return x
    
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
# print(os.listdir("../input"))

import time

In [95]:
class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        
        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp

In [96]:
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model = InceptionV3([block_idx])
model=model.cuda()

In [97]:
def imread(filename):
    """
    Loads an image file into a (height, width, 3) uint8 ndarray.
    """
    image = Image.open(filename)
    img = image.resize((299, 299), Image.ANTIALIAS)
    return np.asarray(img, dtype=np.uint8)[..., :3]


In [98]:
ex = imread(real_images[0])
ex.shape

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_21228\86717952.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((299, 299), Image.ANTIALIAS)


(299, 299, 3)

In [99]:
def get_activations(files, model, batch_size=10, dims=2048,
                    cuda=True, verbose=False):
    """Calculates the activations of the pool_3 layer for all images.
    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : Batch size of images for the model to process at once.
                     Make sure that the number of samples is a multiple of
                     the batch size, otherwise some samples are ignored. This
                     behavior is retained to match the original FID score
                     implementation.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the number
                     of calculated batches is reported.
    Returns:
    -- A numpy array of dimension (num images, dims) that contains the
       activations of the given tensor when feeding inception with the
       query tensor.
    """
    model.eval()

    if batch_size > len(files):
        print(('Warning: batch size is bigger than the data size. Setting batch size to data size'))
        batch_size = len(files)

    pred_arr = np.empty((len(files), dims))

    for i in tqdm(range(0, len(files), batch_size)):
        if verbose:
            print('\rPropagating batch %d/%d' % (i + 1, n_batches),end='', flush=True)
        start = i
        end = i + batch_size
        
        images = np.array([imread(str(f)).astype(np.float32) for f in files[start:end]])
        
        # Reshape to (n_images, 3, height, width)
        images = images.transpose((0, 3, 1, 2))
        images /= 255

        batch = torch.from_numpy(images).type(torch.FloatTensor)
        if cuda:
            batch = batch.cuda()

        pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
        if pred.size(2) != 1 or pred.size(3) != 1:
            pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

        pred_arr[start:end] = pred.cpu().data.numpy().reshape(pred.size(0), -1)

    if verbose:
        print(' done')

    return pred_arr

In [100]:
def calculate_activation_statistics(files, model, batch_size=10,dims=2048, cuda=True, verbose=False):
    """Calculation of the statistics used by the FID.
    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : The images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size
                     depends on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the
                     number of calculated batches is reported.
    Returns:
    -- mu    : The mean over samples of the activations of the pool_3 layer of
               the inception model.
    -- sigma : The covariance matrix of the activations of the pool_3 layer of
               the inception model.
    """
    act = get_activations(files, model, batch_size, dims, cuda, verbose)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

In [101]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.
    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)



In [102]:
def calculate_fretchet(images_real,images_fake,model):
    mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
    mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)

    """get fretched distance"""
    fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
    return fid_value

In [103]:
import glob
real_images = glob.glob('data\\real_images/*')
fake_images = glob.glob('data\\fake_images/*')

In [104]:
real_images

['data\\real_images\\Alan_Tudyk.jpg',
 'data\\real_images\\Alicja_Bachleda.jpg',
 'data\\real_images\\Damian_Lewis.jpg',
 'data\\real_images\\Eoin_Macken.jpg',
 'data\\real_images\\Gilles_Marini.JPG',
 'data\\real_images\\Li_Bingbing.png',
 'data\\real_images\\Moon_Bloodgood.jpg',
 'data\\real_images\\Suraj_Sharma.jpg',
 'data\\real_images\\Witney_Carson.jpg',
 'data\\real_images\\Woody_Harrelson.jpg']

In [105]:
fake_images

['data\\fake_images\\Alan_Tudyk.png',
 'data\\fake_images\\Alicja_Bachleda.png',
 'data\\fake_images\\Bingbing_Li.png',
 'data\\fake_images\\Damian_Lewis.png',
 'data\\fake_images\\Eoin_Macken.png',
 'data\\fake_images\\Gilles_Marini.png',
 'data\\fake_images\\Moon_Bloodgood.png',
 'data\\fake_images\\Suraj_Sharma.png',
 'data\\fake_images\\Witney_Carson.png',
 'data\\fake_images\\Woody_Harrelson.png']

In [106]:
fretchet_dist = calculate_fretchet(real_images,fake_images,model) 
fretchet_dist

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Ahmed\AppData\Local\Temp\ipykernel_21228\86717952.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((299, 299), Image.ANTIALIAS)
100%|██████████| 1/1 [00:00<00:00,  5.09it/s]


342.5705410487059

In [107]:
paper_images = glob.glob('data\\paper_images/*')
paper_images

['data\\paper_images\\Alan_Tudyk.png',
 'data\\paper_images\\Alicja_Bachleda.png',
 'data\\paper_images\\Bingbing_Li.png',
 'data\\paper_images\\Damian_Lewis.png',
 'data\\paper_images\\Eoin_Macken.png',
 'data\\paper_images\\Gilles_Marini.png',
 'data\\paper_images\\Moon_Bloodgood.png',
 'data\\paper_images\\Suraj_Sharma.png',
 'data\\paper_images\\Witney_Carson.png',
 'data\\paper_images\\Woody_Harrelson.png']

In [108]:
fretchet_dist = calculate_fretchet(real_images,paper_images,model) 
fretchet_dist

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Ahmed\AppData\Local\Temp\ipykernel_21228\86717952.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((299, 299), Image.ANTIALIAS)
100%|██████████| 1/1 [00:00<00:00,  3.40it/s]


318.80693291057815

In [109]:
fretchet_dist = calculate_fretchet(real_images,real_images,model) 
fretchet_dist

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Ahmed\AppData\Local\Temp\ipykernel_21228\86717952.py:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = image.resize((299, 299), Image.ANTIALIAS)
100%|██████████| 1/1 [00:00<00:00,  3.29it/s]


-0.0001528600594724594